In [1]:
import pandas as pd
from sqlalchemy import create_engine
import regex as re
import time
from datetime import datetime
import os
import glob

In [2]:
all_data = pd.DataFrame()
allExcelFiles =glob.glob("/home/trishalaswain/ClientData/PH/PH_JULY_HALF/PH/*31.tsv")
chino_all_overall_sheets_combined = pd.concat(pd.read_table(excelFile, index_col=False) for excelFile in allExcelFiles)


In [6]:
chino_all_overall_sheets_combined.reset_index().to_excel('/home/trishalaswain/ClientData/PH/PH_JULY_HALF/PH_JULY_ALL_SHEET.xlsx')

In [ ]:
chino_all_overall_sheets_combined = pd.read_excel('/home/trishalaswain/Documents/CHINO_DATA_JUNE_TILL_25th.xlsx', index_col=False)

In [ ]:
chino_all_overall_sheets_combined.columns

In [ ]:
# cols_to_use = ['id', 'serviceDate', 'processingDate', 'lastCodingDate', 'month', 'accountNumber', 'encDocIds', 'EANDM_Final', 'EANDM_Accepted', 'EANDM_Rejected', 'EANDM_Added', 'EANDM_Suggested', 'EANDM_Accepted_Count', 'EANDM_Rejected_Count', 'EANDM_Added_Count']

In [ ]:
# chino_all_overall_sheets_combined.columns

In [ ]:
# filtering accounts with single document
# chino_all_overall_sheets_combined = chino_all_overall_sheets_combined[~chino_all_overall_sheets_combined.encDocIds.str.contains(',')]
# chino_all_overall_sheets_combined.shape
chino_all_overall_sheets_combined.local_output_3 = chino_all_overall_sheets_combined.local_output_3.astype('str')
# chino_all_overall_sheets_combined.dropna(inplace=True)
chino_all_overall_sheets_combined = chino_all_overall_sheets_combined[~chino_all_overall_sheets_combined.local_output_3.str.contains('nan')]

In [ ]:
# applying filter of lastProcessingDate >= 01-01-2023
# chino_all_overall_sheets_combined['serviceDate'] = pd.to_datetime( chino_all_overall_sheets_combined.serviceDate)
# chino_all_overall_sheets_combined = chino_all_overall_sheets_combined[chino_all_overall_sheets_combined.serviceDate >= '2023-01-01']

In [ ]:
chino_all_overall_sheets_combined

In [ ]:
chino_all_overall_sheets_combined = chino_all_overall_sheets_combined[cols_to_use]
chino_all_overall_sheets_combined.reset_index(inplace=True)
chino_all_overall_sheets_combined

In [ ]:
for i,r in chino_all_overall_sheets_combined.iterrows():
    chino_all_overall_sheets_combined.loc[i,'clean_suggested'] = ', '.join([str(elem) for elem in re.findall(r"\b\d{5}\b", str(r['suggested']))])
    chino_all_overall_sheets_combined.loc[i,'clean_final'] = ', '.join([str(elem) for elem in re.findall(r"\b\d{5}\b", str(r['final']))])

In [ ]:
# df_custom_cpt_filtered = pd.read_table('/home/trishalaswain/ClientData/PH/CHINO_MONTH_WISE/CHINO_ALL_OP/devansh_chino.tsv', index_col=False)

In [ ]:
# df_custom_cpt_accounts_filtered = df_custom_cpt_filtered.accountNumber.to_list()

In [ ]:
# type(df_custom_cpt_accounts_filtered[0])

In [ ]:
# chino_all_overall_sheets_combined.reset_index(inplace=True)

In [ ]:
# for i, row in chino_all_overall_sheets_combined.iterrows():
#     if str(row['accountNumber']) not in df_custom_cpt_accounts_filtered:
#         chino_all_overall_sheets_combined.drop([i], inplace=True)
# chino_all_overall_sheets_combined.reset_index(inplace=True)

In [ ]:
chino_all_overall_sheets_combined

In [ ]:
# chino_all_overall_sheets_combined.EANDM_Final = chino_all_overall_sheets_combined.EANDM_Accepted.apply(lambda x:chino_all_overall_sheets_combined(x))+chino_all_overall_sheets_combined.EANDM_Added.apply(lambda x:chino_all_overall_sheets_combined(x))
# chino_all_overall_sheets_combined.EANDM_Suggested = chino_all_overall_sheets_combined.EANDM_Accepted.apply(lambda x:chino_all_overall_sheets_combined(x))+chino_all_overall_sheets_combined.EANDM_Rejected.apply(lambda x:chino_all_overall_sheets_combined(x))

In [ ]:
from collections import OrderedDict
def get_accuracy(data_df, codetype):
    accept = data_df[codetype+'_Accepted_Count'].sum()
    reject = data_df[codetype+'_Rejected_Count'].sum()
    add = data_df[codetype+'_Added_Count'].sum()
    
    d = OrderedDict()
    
    d['encounters'] = data_df.shape[0]
    
    d[codetype+'_accepted'] = accept
    d[codetype+'_rejected'] = reject
    d[codetype+'_added'] = add
    
    d[codetype+'_billed'] = accept+add
    
    if accept == 0:
        if add + reject == 0:
            d['precision'] = 1
            d['recall'] = 1
            d['f1'] = 1
        else:
            d['precision'] = 0.0
            d['recall'] = 0.0
            d['f1'] = 0.0
    else:
        d['precision'] = accept / (accept + reject)
        d['recall'] = accept / (accept + add)
        d['f1'] = 2 * d['precision'] * d['recall'] / (d['precision'] + d['recall'])
    
    return pd.Series(d)

In [ ]:
chino_all_overall_sheets_combined.clean_final.value_counts()

In [ ]:
chino_all_overall_sheets_combined.clean_suggested.value_counts()

In [ ]:
# valid_encounter_ids = chino_all_overall_sheets_combined.id.to_list()

In [ ]:
# filtering multiple documents in singke encounter
import shutil
encounter_downloaded = []
dir_path = '/home/trishalaswain/ClientData/PH/CHINO_MONTH_WISE/CHINO_COMBINED_EMCS (copy)'
deleted_file_count = 0
for path in os.listdir(dir_path):
    sub_dir = os.path.join(dir_path, path)
    # print(path)
    
    if str(path) not in str(valid_encounter_ids):
        
        # print(sub_dir)
        deleted_file_count += 1
        shutil.rmtree(sub_dir)
    else:
        encounter_downloaded.append(str(path))
    # for file in os.listdir(sub_dir):
    #     count += 1
    # if count > 2:
    #     print('Deleting multiple docs:', sub_dir)
    #     shutil.rmtree(sub_dir)
    #     deleted_file_count += 1
    # else:
    #     encounter_downloaded.append(str(path))
print('Deleted encounter counts: ',deleted_file_count)        

In [ ]:
dir_path = '/home/trishalaswain/ClientData/PH/CHINO_MONTH_WISE/CHINO_COMBINED_EMCS (copy)'
deleted_file_count = 0
for path in os.listdir(dir_path):
    sub_dir = os.path.join(dir_path, path)
    count = 0
    for file in os.listdir(sub_dir):
        count += 1
        if count > 2:
            print('Deleting multiple docs:', sub_dir)
            # shutil.rmtree(sub_dir)
            deleted_file_count += 1
print(deleted_file_count)       

In [ ]:
rows_to_delete = 0
for i, r in chino_all_overall_sheets_combined.iterrows():
    if str(r['id']) not in str(encounter_downloaded):
        rows_to_delete += 1
        chino_all_overall_sheets_combined.drop([i], inplace=True)
print('rows_to_delete', rows_to_delete)

In [ ]:
# chino_all_overall_sheets_combined

In [ ]:
chino_all_overall_sheets_combined.shape

In [ ]:
acc = chino_all_overall_sheets_combined.groupby(['month']).apply(get_accuracy, 'EANDM').sort_values(['f1','EANDM_rejected'],ascending=[True,False]).reset_index()
acc

In [ ]:
acc.to_excel('/home/trishalaswain/acc.xlsx')

In [ ]:
# local_output_dataframe = pd.read_excel('/home/trishalaswain/ClientData/PH/CHINO_MONTH_WISE/CHINO_LOCAL_OUTPUT_AS_ON_JUNE_9.xlsx', index_col=False)

In [ ]:
chino_all_overall_sheets_combined.accountNumber = chino_all_overall_sheets_combined.accountNumber.astype('str')

In [ ]:
chino_all_overall_sheets_combined.shape

In [ ]:
# chino_all_overall_sheets_combined.reset_index(inplace=True)

In [ ]:
# chino_all_overall_sheets_combined.to_excel('/home/trishalaswain/chino_all_overall_sheets_combined_FINAL.xlsx')

In [ ]:
new_chino_all_overall_sheets_combined = pd.read_excel('/home/trishalaswain/chino_all_overall_sheets_combined_FINAL.xlsx', index_col=False)

In [ ]:
# for i, r in local_output_dataframe.iterrows():
#     chino_all_overall_sheets_combined.loc[chino_all_overall_sheets_combined.accountNumber==str(r['account_number']), 'local_suggested'] = str(r['local_output'])

In [ ]:
# new_chino_all_overall_sheets_combined_2 = new_chino_all_overall_sheets_combined.copy()

In [ ]:
new_chino_all_overall_sheets_combined.clean_final.value_counts()

In [ ]:
    # x=str(x)
    # new_code=''
    # if re.match('.*9921[1-5].*',x) and re.match('.*9920[1-5].*',x):
    #         codes=x.split(',')
    #         for code in codes:
    #             if re.match('.*9921[1-5].*', code):
    #                 new_code=code.strip()  

In [ ]:
# for i, r in new_chino_all_overall_sheets_combined.iterrows():
    
#     if re.match('.*9922[1-5].*',str(r['clean_final'])) and re.match('.*9928[1-5].*',str(r['clean_final'])):
#         # codes = str(r['clean_final']).split(',')
        
#         for code in codes:
#             obv_code = re.findall('9922[1-5].*',code)
            
#             if '9' in str(obv_code):
#                 code_to_remove = str(obv_code).replace("['", "").replace("']", "")
#                 print(str(obv_code))
#                 # codes=codes.remove(code_to_remove)
        
#         # print(codes)
#         # new_chino_all_overall_sheets_combined.loc[i,'clean_final'] = r['clean_final'].remove(code)
new_chino_all_overall_sheets_combined.local_suggested.value_counts()

In [ ]:
# for i,r in new_chino_all_overall_sheets_combined.iterrows():
#     new_chino_all_overall_sheets_combined.loc[i, 'local_suggested_set'] = r['local_suggested'].split(', ')
# new_chino_all_overall_sheets_combined.local_suggested_set = new_chino_all_overall_sheets_combined.local_suggested_set.astype('str')

In [ ]:
# new_chino_all_overall_sheets_combined_2.reset_index(inplace=True)
# for i,r in new_chino_all_overall_sheets_combined_2.iterrows():
#     if str(new_chino_all_overall_sheets_combined_2.loc[i, 'clean_final']) == '99418, 99223':
#         new_chino_all_overall_sheets_combined_2.loc[i, 'clean_final'] = '99223'
        
#     if str(new_chino_all_overall_sheets_combined_2.loc[i, 'clean_final']) == '99284,99222':
#         new_chino_all_overall_sheets_combined_2.loc[i, 'clean_final'] = '99222'
        
#     if str(new_chino_all_overall_sheets_combined_2.loc[i, 'clean_final']) == '99285,99223':
#         new_chino_all_overall_sheets_combined_2.loc[i, 'clean_final'] = '99223'
        
#     if str(new_chino_all_overall_sheets_combined_2.loc[i, 'clean_final']) == '99239,99223':
#         new_chino_all_overall_sheets_combined_2.loc[i, 'clean_final'] = '99223'
        
#     if str(new_chino_all_overall_sheets_combined_2.loc[i, 'clean_final']) == '99285,99236':
#         new_chino_all_overall_sheets_combined_2.loc[i, 'clean_final'] = '99285'
        
#     if str(new_chino_all_overall_sheets_combined_2.loc[i, 'clean_final']) == '99285,99213':
#         new_chino_all_overall_sheets_combined_2.loc[i, 'clean_final'] = '99285'
        
#     if str(new_chino_all_overall_sheets_combined_2.loc[i, 'clean_final']) == '99292,99223':
#         new_chino_all_overall_sheets_combined_2.loc[i, 'clean_final'] = '99292'
#         # 
        
    # if str(new_chino_all_overall_sheets_combined_2.loc[i, 'clean_suggested']) == '99222':
    #     new_chino_all_overall_sheets_combined_2.loc[i, 'clean_suggested'] = '99284'

    # if str(new_chino_all_overall_sheets_combined_2.loc[i, 'clean_suggested']) == '99223':
    #     new_chino_all_overall_sheets_combined_2.loc[i, 'clean_suggested'] = '99285'

    # if str(new_chino_all_overall_sheets_combined_2.loc[i, 'clean_suggested']) == '99223,99406':
    #     new_chino_all_overall_sheets_combined_2.loc[i, 'clean_suggested'] = '99284, 99406'

    # if str(new_chino_all_overall_sheets_combined_2.loc[i, 'clean_suggested']) == '99222,99406':
    #     new_chino_all_overall_sheets_combined_2.loc[i, 'clean_suggested'] = '99284,99406'

    # if str(new_chino_all_overall_sheets_combined_2.loc[i, 'clean_suggested']) == '99221':
    #     new_chino_all_overall_sheets_combined_2.loc[i, 'clean_suggested'] = ''

    # if str(new_chino_all_overall_sheets_combined_2.loc[i, 'clean_suggested']) == '99418,99223':
    #     new_chino_all_overall_sheets_combined_2.loc[i, 'clean_suggested'] = '99285'
        
    # if str(new_chino_all_overall_sheets_combined_2.loc[i, 'clean_suggested']) == '99236':
    #     new_chino_all_overall_sheets_combined_2.loc[i, 'clean_suggested'] = '99285'
        
    # if str(new_chino_all_overall_sheets_combined_2.loc[i, 'clean_suggested']) == '99418,99236':
    #     new_chino_all_overall_sheets_combined_2.loc[i, 'clean_suggested'] = '99418,99285'
        
    # 

In [ ]:
def get_counts(x):
    
    suggested = str(x['clean_suggested'])
    final = str(x['clean_final'])
    
    if suggested != '':
        suggested_set = set(suggested.split(', '))
        final_set = set(final.split(', '))

        if '9' not in suggested:
            suggested_set.clear()
            # final_set.clear()
        if '9' not in final:
            final_set.clear()
        # if '99406' in suggested:
        #     print(suggested)
        #     suggested_set.remove('99406')

            
        # if '99418' in suggested:
        #     print(suggested)
        #     suggested_set.remove('99418')
            
        # if '9922' in suggested:
        #     suggested_set.clear()
        #     final_set.clear()
        
        # if '9922' in suggested:
        #     suggested_set.clear()
        #     final_set.clear()
        
    
        accepted = len(suggested_set.intersection(final_set))
        rejected = len(suggested_set - final_set)
        added = len(final_set - suggested_set)
        if (rejected >= 1 or added >= 1):
            print('rejected',rejected)
            print('added',added)
            print('suggested_set',suggested_set,'\t final_set',final_set)
            print('accountNumber', str(x['account_number']))

        return pd.Series({'NEW_EANDM_Accepted_Count':accepted, 'NEW_EANDM_Rejected_Count':rejected, 'NEW_EANDM_Added_Count':added})

In [ ]:
# count_df =  new_chino_all_overall_sheets_combined[new_chino_all_overall_sheets_combined.month=='2023-05'].apply(get_counts, axis=1)
chino_all_overall_sheets_combined.columns

In [ ]:
count_df =  chino_all_overall_sheets_combined.apply(get_counts, axis=1)

In [ ]:
# chino_all_overall_sheets_combined
count_df

In [ ]:
count_df.NEW_EANDM_Accepted_Count.sum(), count_df.NEW_EANDM_Rejected_Count.sum(), count_df.NEW_EANDM_Added_Count.sum()

In [ ]:
precision = count_df.NEW_EANDM_Accepted_Count.sum() / (count_df.NEW_EANDM_Accepted_Count.sum() + count_df.NEW_EANDM_Rejected_Count.sum())
recall = count_df.NEW_EANDM_Accepted_Count.sum() / (count_df.NEW_EANDM_Accepted_Count.sum() + count_df.NEW_EANDM_Added_Count.sum())
f1_score = (2 * precision * recall) / (recall + precision)

In [ ]:
precision,recall, f1_score

In [ ]:
new_chino_all_overall_sheets_combined_3 = pd.concat([new_chino_all_overall_sheets_combined, count_df], axis = 1)
# new_chino_all_overall_sheets_combined_3.to_excel('/home/trishalaswain/new_chino_all_overall_sheets_combined_3.xlsx')

In [ ]:
data = {"precision":precision, "recall":recall, "f1_score":f1_score}
pd.Series(data).to_frame('new_overall_sheet_stats_local')

In [ ]:
# new_chino_all_overall_sheets_combined_3 = new_chino_all_overall_sheets_combined_3[new_chino_all_overall_sheets_combined_3.month=='2023-05']
# new_chino_all_overall_sheets_combined_3

In [ ]:
new_chino_all_overall_sheets_combined_3.NEW_EANDM_Accepted_Count.sum(), new_chino_all_overall_sheets_combined_3.NEW_EANDM_Rejected_Count.sum(), new_chino_all_overall_sheets_combined_3.NEW_EANDM_Added_Count.sum()

In [ ]:
from collections import OrderedDict
def get_code_counts(data_df, codetype):
    accept = data_df[codetype+'_Accepted_Count'].sum()
    reject = data_df[codetype+'_Rejected_Count'].sum()
    add = data_df[codetype+'_Added_Count'].sum()
    
    d = OrderedDict()
    d['encounters'] = data_df.shape[0]
    d['total_accepted_counts'] = accept
    d['total_rejected_counts'] = reject
    d['total_added_counts'] = add

    d['precision'] = accept / (accept + reject)
    d['recall'] = accept / (accept + add)
    d['f1'] = 2 * d['precision'] * d['recall'] / (d['precision'] + d['recall'])

    return pd.Series(d)

In [ ]:
from collections import OrderedDict
def get_accuracy(data_df, codetype):
    accept = data_df[codetype+'_Accepted_Count'].sum()
    reject = data_df[codetype+'_Rejected_Count'].sum()
    add = data_df[codetype+'_Added_Count'].sum()
    
    d = OrderedDict()
    
    d['encounters'] = data_df.shape[0]
    
    d[codetype+'_accepted'] = accept
    d[codetype+'_rejected'] = reject
    d[codetype+'_added'] = add
    
    d[codetype+'_billed'] = accept+add
    
    if accept == 0:
        if add + reject == 0:
            d['precision'] = 1
            d['recall'] = 1
            d['f1'] = 1
        else:
            d['precision'] = 0.0
            d['recall'] = 0.0
            d['f1'] = 0.0
    else:
        d['precision'] = accept / (accept + reject)
        d['recall'] = accept / (accept + add)
        d['f1'] = 2 * d['precision'] * d['recall'] / (d['precision'] + d['recall'])
    
    return pd.Series(d)

In [ ]:
new_chino_all_overall_sheets_combined_2_accuracy = new_chino_all_overall_sheets_combined_3.groupby(['month']).apply(get_code_counts, 'NEW_EANDM').reset_index()
# new_chino_all_overall_sheets_combined_2_accuracy.to_excel('/home/trishalaswain/new_chino_all_overall_sheets_combined_2_accuracy.xlsx')
new_chino_all_overall_sheets_combined_2_accuracy

In [ ]:
chino_all_overall_sheets_combined_acc  = new_chino_all_overall_sheets_combined.groupby(['month']).apply(get_accuracy, 'EANDM').reset_index()
# chino_all_overall_sheets_combined_acc.to_excel('/home/trishalaswain/chino_all_overall_sheets_combined_acc1.xlsx')
chino_all_overall_sheets_combined_acc

In [ ]:
new_chino_all_overall_sheets_combined.columns

In [ ]:
new_chino_all_overall_sheets_combined_may = new_chino_all_overall_sheets_combined[new_chino_all_overall_sheets_combined.month=='2023-05']

In [ ]:
cs =  pd.crosstab(new_chino_all_overall_sheets_combined.clean_final, new_chino_all_overall_sheets_combined.clean_suggested, margins=True)

In [ ]:
# cs.to_excel('/home/trishalaswain/crosstab_2.xlsx')
new_chino_all_overall_sheets_combined_may.local_suggested_new = new_chino_all_overall_sheets_combined_may.local_suggested_new.astype('str')
new_chino_all_overall_sheets_combined_may.clean_suggested = new_chino_all_overall_sheets_combined_may.clean_suggested.astype('str')

new_chino_all_overall_sheets_combined_may.clean_final = new_chino_all_overall_sheets_combined_may.clean_final.astype('str')
new_chino_all_overall_sheets_combined_may.local_suggested.value_counts()

In [ ]:
new_chino_all_overall_sheets_combined_may['local_suggested_new'].str.contains('99284').sum()

In [ ]:
from operator import length_hint


code_wise_count = pd.DataFrame()
code_wise_count['code']=None


code_list = ['99213',
'99222',
'99223',
'99236',
'99239',
'99282',
'99283',
'99284',
'99285',
'99291',
'99292',
'99406',
'99418']
def get_codewise_counts(code_list, df_to_use):
    code_wise_count = pd.DataFrame(code_list, columns=['code'])
    code_wise_count['clean_suggested']=0
    code_wise_count['final']=0  
    code_wise_count.set_index('code', inplace=True)
        
    # code_wise_count
    for code in code_list:
        # code = str(code)
        # print(code)
        code_wise_count.loc[code,'clean_suggested'] += df_to_use['clean_suggested'].str.contains(code).sum()
        code_wise_count.loc[code, 'final'] += df_to_use['clean_final'].str.contains(code).sum()
        
        
    return code_wise_count

In [ ]:
get_codewise_counts(code_list, chino_all_overall_sheets_combined).to_excel('/home/trishalaswain/ClientData/PH/CHINO_MONTH_WISE/code_wise_counts.xlsx')

In [ ]:
get_codewise_counts(code_list, new_chino_all_overall_sheets_combined_may)

In [ ]:
new_chino_all_overall_sheets_combined['clean_suggested'].str.contains('99284').sum()